# EP1 - Conjunto de Maldelbrot e paralelização com pThreads e OpenMP

| Nome | NUSP |
|------|------|
| Giulia C. de Nardi | 10692203 |
| Vitor D. Tamae | 10705620 |
| Lucy Anne de Omena Evangelista | 11221776 |
| Leonardo Costa Santos | 10783142 |
| Alexandre Muller Jones | 8038149 |


INFORMAÇÔES SOBRE O DESENVOLVIMENTO DO RELATÓRIO:

- **Para o caso dos mandelbrot paralelizados, podemos ter dois tempos como saída: o considerando o tempo de alocação de memória e I/O, e o tempo não considerando esses fatores. Ainda não implementado para a realização das funções, mas de fácil ajuste nas funções feitas neste notebook.**


## Configuração do ambiente

Atualizando os pacotes Julia

In [ ]:
] up

Verificando o status dos pacotes, e se há algum problema, com o comando:

In [ ]:
] st

## Funções para realização dos experimentos

In [ ]:
; make mandelbrot_seq

In [4]:
; ./mandelbrot_seq

usage: ./mandelbrot_seq c_x_min c_x_max c_y_min c_y_max image_size
examples with image_size = 11500:
    Full Picture:         ./mandelbrot_seq -2.5 1.5 -2.0 2.0 11500
    Seahorse Valley:      ./mandelbrot_seq -0.8 -0.7 0.05 0.15 11500
    Elephant Valley:      ./mandelbrot_seq 0.175 0.375 -0.1 0.1 11500
    Triple Spiral Valley: ./mandelbrot_seq -0.188 -0.012 0.554 0.754 11500


In [5]:
; ./mandelbrot_seq 0.175 0.375 -0.1 0.1 200 0

0.040757


A função abaixo recebe parâmetros `size`, com o tamanho da imagem, `f`, com a id do tupo de fractal a ser feito (0 - Full, 2 - Triple Spiral, 3 - Elephant, 4 - Seahorse), `mandel`, com o nome da função a ser executada (`./mandelbrot_seq`, `./mandelbrot_opm`, `./mandelbrot_pth`),e `threads`, com o número de threads do programa paralelo. A função executa o programa `mandelbrot` com os parâmetros dados e devolve um `DataFrame` com os resultados.

In [18]:
using DataFrames, Query, StatsPlots, Statistics

function measure_mandelbrot(size, f, mandel; thread = 0)    
    if f == 0  mode = `-2.5 1.5 -2.0 2.0` #full
    elseif f == 1  mode = `-0.188 -0.012 0.554 0.754` #triple spiral
    elseif f == 2  mode = `0.175 0.375 -0.1 0.1` # elephant
    elseif f == 3  mode = `-0.8 -0.7 0.05 0.15` #seahorse
    end
    
    if thread != 0 
    results = parse.(Float64,
        read(`./$mandel $mode $size $thread`, String))
    else
    results = parse.(Float64,
        read(`./$mandel $mode $size`, String))
    end
        
    return DataFrame(size = size,
        f = f,
        threads = thread,
        duration = results[1])
end

┌ Info: Precompiling StatsPlots [f3b207a7-027a-5e70-b257-86293d7955fd]
└ @ Base loading.jl:1273


measure_mandelbrot (generic function with 1 method)

A função `run_experiments` recebe os mesmos parâmetros `size`, `f`,`mandel` e `threads`, e um parâmetro adicional `repetitions`, com o número de repetições de cada experimento com um dado número de `threads`. A função devolve um `DataFrame` com todos os experimentos.

In [19]:
function run_experiments(size, f, mandel, repetitions; threads = [])
    run(`make $mandel`)
    
    results = DataFrame(size = Int[],
        f = Int[],
        threads = Int[],
        duration = Float64[]) 
    
    if threads != []
    for t in threads
        for s in size
        for r in 1:repetitions
            append!(results,
                measure_mandelbrot(s, f, mandel, thread = t))    
        end
        end
    end
    else
        for r in 1:repetitions
        for s in size
            append!(results,
                measure_mandelbrot(s, f, mandel))    
        end
        end
    end 
    
    return results
end

run_experiments (generic function with 1 method)

A função `parse_results` recebe um `DataFrame` de resultados, produzido pela função `run_experiments`. A função devolve um `DataFrame` com a média e o intervalo de confiança da média a 95% dos tempos de execução, agrupados por número de threads.

In [20]:
function parse_results(results)
    parsed_results = results |>
                    @groupby({_.threads,_.size}) |>
                    @map({threads = key(_).threads,
                          size = _.size[1],
                          mean_duration = mean(_.duration),
                          ci_duration = 1.96 * std(_.duration)}) |>
                    DataFrame
    
    return parsed_results
end

parse_results (generic function with 1 method)

## Funções para traçar gráficos

A função abaixo permite que sejam traçadas até 5 séries de dados em um mesmo gráfico do tipo scatter.

In [21]:
pgfplotsx()

function plot_results(x, y, series_label, yerror; y2 = [], series_label2 = [], yerror2 = [], 
        y3 = [], series_label3 = [], yerror3 = [], y4 = [], series_label4 = [], yerror4 = [],
        y5 = [], series_label5 = [], yerror5 = [])
    max_thread_power = 5
    
    p = scatter(x, y, xaxis = :log2, xlabel = "Threads", xticks = [2 ^ x for x in 0:max_thread_power],
        yerror = yerror, alpha = 0.6, 
        labels = series_label, legend = :bottomright)
    
    if y2 != []
        p = scatter!(x, y2, xaxis = :log2, xticks = [2 ^ x for x in 0:max_thread_power],
            yerror = yerror2, alpha = 0.6,
            labels = series_label2, legend = :bottomright)
    end
    if y3 != []
        p = scatter!(x, y3, xaxis = :log2, xticks = [2 ^ x for x in 0:max_thread_power],
            yerror = yerror3, alpha = 0.6,
            labels = series_label3, legend = :bottomright)
    end
    if y4 != []
        p = scatter!(x, y4, xaxis = :log2, xticks = [2 ^ x for x in 0:max_thread_power],
            yerror = yerror4, alpha = 0.6,
            labels = series_label4, legend = :bottomright)
    end
    if y5 != []
        p = scatter!(x, y5, xaxis = :log2, xticks = [2 ^ x for x in 0:max_thread_power],
            yerror = yerror5, alpha = 0.6,
            labels = series_label5, legend = :bottomright)
    end
    
    return p
end

┌ Info: Precompiling PGFPlotsX [8314cec4-20b6-5062-9cdb-752b83310925]
└ @ Base loading.jl:1273


plot_results (generic function with 1 method)

## Condições para os experimentos

In [22]:
size = [2 ^ x for x in 4:13]
thread = [2 ^ x for x in 0:5]
repetitions = 10;

## Mandelbrot sequencial

Nesta parte, traremos a execução do maldelbrot em sua versão sequencial, junto com a análise de tempo de execução para os diferentes tipo de fractais (Triple Spiral, Elephant, Seahorse & Full) e em diferentes resoluções ($ 2^4 \cdots 2^{13}$)

Realizando as medições para o mandelbrot sequencial:

In [23]:
results = run_experiments(size, 0, "mandelbrot_seq", repetitions)
seq_full = parse_results(results)

make: 'mandelbrot_seq' is up to date.


,threads,size,mean_duration,ci_duration
,Int64,Int64,Float64,Float64
1,0,16,5.36e-5,1.62993e-5
2,0,32,0.0001936,5.58911e-5
3,0,64,0.0007408,0.000162279
4,0,128,0.0031887,0.00227842
5,0,256,0.0116996,0.00322845
6,0,512,0.0446544,0.0125451
7,0,1024,0.174277,0.0377931
8,0,2048,0.682131,0.121173
9,0,4096,2.67757,0.420568


In [ ]:
results = run_experiments(size, 1, "mandelbrot_seq", repetitions)
seq_triplespiral = parse_results(results);

In [ ]:
results = run_experiments(size, 2, "mandelbrot_seq", repetitions)
seq_elephant = parse_results(results);

In [ ]:
results = run_experiments(size, 3, "mandelbrot_seq", repetitions)
seq_seahorse = parse_results(results);

## Mandelbrot com pthreads

In [ ]:
results = run_experiments(size, 0, "mandelbrot_pth", repetitions,threads=thread)
pth_full = parse_results(results)

make: 'mandelbrot_pth' is up to date.


In [ ]:
results = run_experiments(size, 1, "mandelbrot_pth", repetitions,threads=thread)
pth_triplespiral = parse_results(results);

In [ ]:
results = run_experiments(size, 2, "mandelbrot_pth", repetitions,threads=thread)
pth_elephant = parse_results(results);

In [ ]:
results = run_experiments(size, 3, "mandelbrot_pth", repetitions,threads=thread)
pth_seahorse = parse_results(results);

## Mandelbrot com OpenMP

In [ ]:
results = run_experiments(size, 0, "mandelbrot_omp", repetitions,threads=thread)
omp_full = parse_results(results)

In [ ]:
results = run_experiments(size, 1, "mandelbrot_omp", repetitions,threads=thread)
omp_triplespiral = parse_results(results);

In [ ]:
results = run_experiments(size, 2, "mandelbrot_omp", repetitions,threads=thread)
omp_elephant = parse_results(results);

In [ ]:
results = run_experiments(size, 3, "mandelbrot_omp", repetitions,threads=thread)
omp_seahorse = parse_results(results);

## Gráficos comparativos

Ao final, teremos os dataframes:

|Dataframe | Full | Triple Spiral | Seahorse |
|----------|--------|--------|--------|
|Sequencial|seq_full|seq_triplespiral|seq_seahorse|
|PThreads|pth_full|pth_triplespiral|pth_seahorse|
|OpenMP|omp_full|omp_triplespiral|omp_seahorse|

Realizaremos os gráficos a partir de partições do dataframe, como mostrados abaixo:

In [ ]:
filter(row -> row[:threads] == 1, parsed_results)

In [ ]:
filter(row -> row[:size] == 16, parsed_results)

In [ ]:
filter(row -> row[:size] == 16, parsed_results).mean_duration

### Comparando desempenho por tamanho da imagem

Ideias para os gráficos: 
> Comparar desempenho por tamanho da imagem ( 5 grafos > para tamanhos das imagens. Cada serie no grafico deve ser uma forma gerar a imagem)

> Comparar desempenho por tipo de gráfico produzido ( 3 x 4 gráficos, 4 áreas com 3 tipos de calculo cada / cruzar com tamanho da entrada também? daí seriam 4x 3 x 10)

> estou confouzer, me ajudem a saber quais gráficos fazer

In [ ]:
plot_results(
    filter(row -> row[:size] == 16, parsed_results).threads,
    filter(row -> row[:size] == 16, parsed_results).mean_duration,    
    "16", filter(row -> row[:size] == 16, parsed_results).ci_duration,
    y2 = filter(row -> row[:size] == 32, parsed_results).mean_duration,
    series_label2 = "32", yerror2 = filter(row -> row[:size] == 32, parsed_results).ci_duration)

### Comparando desempenho por quantidade de threads